## Introduction: the acoustic  particulate cylinder

The propagation of waves in free space is governed by the 2D Helmholtz equation. We denote by $\mathrm V_n$ and $\mathrm U_n$ the cylindrical Bessel functions

$$
\tag{1}
 \mathrm V_n(k\mathbf r):=\mathrm J_n(kr)\mathrm e^{\mathrm in\theta} 
\quad\text{and}\quad
\mathrm U_n(k \mathbf r):= \mathrm H_n(k r)\mathrm e^{\mathrm in\theta}
$$

where $k$ is the background wavenumber and  $(r,\theta)$ are the polar coordinates of $\mathbf r$, ie $\mathbf r = (r\cos \theta,r\sin\theta)$, $\mathrm J_n$ and $\mathrm H_n$ are respectively the Bessel function of order zero and the Hankel function. 

We consider the scattering from a set of $J$ cylinders, referred to as particles, confined in a circular area of radius $R$. The  incident field $u_i$ can be decomposed in modes:

$$
\tag{2}
u_i(\mathbf r) =  \sum_{n=-\infty}^{+\infty}g_n\mathrm V_n(k\mathbf r),\quad g_n\in\mathbb{C}.
$$

Then the averaged scattered field $\langle u_s\rangle$ (over all possible configurations of the $J$ particles confined in the disc of radius $R$) can be written

$$
\tag{3}
\langle u_s\rangle(\mathbf r) =  \sum_{n=-\infty}^{+\infty} \langle \mathfrak F_n \rangle\mathrm H_n(k\mathbf r), \quad  \langle \mathfrak F_n \rangle = \mathrm T_ng_n
$$

where $\mathrm T_n\in\mathbb{C}$ is related to the effective T-matrix $\mathrm T_{n,m}$ of the effective cylinder of radius $R$. More precisely,

$$
\tag{4}
\mathrm T_{n,m} := \delta_{n,m} \mathrm T_n.
$$

With this definition, the scattering from the effective cylinder of radius $R$ can be summerized in matrix form 

$$
\tag{5}
 \langle \mathbf{F} \rangle = \mathbf T \mathbf g
$$

where we defined the vectors $\langle \mathbf{F} \rangle=( \langle \mathfrak F_n \rangle)_n$ and $\mathbf g = (g_n)_n$.

## Example 3: The mode to mode scattering


The goal of this section is to clarify how the function run_MC_validation! presented in the previous Example 2. computes the coefficients $\mathrm T_n$ of the effective T-matrix.

### 3.1) Radial scattering

#### 3.1.a) Radial source

For a 2D setting, a modal source is of the form 

$$
\tag{6}
u_i(\mathbf{r}) = \mathrm V_n(\mathbf{kr})
$$

where $\mathrm V_n(\mathbf{kr})$ is defined by (Eq. 1). We first show an example with a radial source which is a specific modal source with $n=0$

In [ ]:
using EffectiveTMatrix
using Plots

dimension=2;                                       
host_medium = Acoustic(dimension; ρ=1.0, c=1.0);   # 2D acoustic problem
input_mode = 0;                                          # mode n=0 
source = mode_source(host_medium,input_mode); # V_0

To plot the source field, we need to set a frequency $\omega$, define a box where to plot the field and choose a resolution

In [ ]:
ω=0.1;                                    # frequency
M=N=20;  
resolution = 100                                 # sizes of the rectangle where to plot
bottomleft = [-M;-N]; topright = [M;N];
region = Box([bottomleft, topright]);
plot(source,ω;bounds=region,res=resolution)

<p align="center">
    <img
    src="source_mode_0.png"
    alt="Alt text"
    title=""
    style="display: inline-block; margin: 0 auto; max-width: 200px">
</p>

#### 3.1.b) Scattering from one configuration of particles

We first need to define the parameters for particles configurations:


In [ ]:
# parameters for particles configurations
radius_big_cylinder = 20.0;                                 # radius of cylinder where particles are confined 
particle = Particle(Acoustic(2; ρ=Inf, c=Inf),Circle(1.0)); # particles type: sound hard particles of radius 1
sp_MC = Specie(particle; volume_fraction = .05);            # # volume fraction is the density of particles

sp_MC then contains the statistics of the particles configuration, a specific configuration of particles is drawn with the function renew_particle_configurations as follows:

In [ ]:
particles_realisation = renew_particle_configurations(sp_MC,radius_big_cylinder);
plot(particles_realisation)

<p align="center">
    <img
    src="particles_configuration.png"
    alt="Alt text"
    title="Particles configuration"
    style="display: inline-block; margin: 0 auto; max-width: 200px">
</p>

We can now compute the scattered field

In [ ]:
basis_order=5;
sim = FrequencySimulation(particles_realisation,source);
scattered_field = run(sim,region,[ω];only_scattered_waves=true,basis_order=basis_order,res=resolution);

plot(scattered_field,ω; field_apply=real,seriestype = :contour,c=:balance) 

<p align="center">
    <img
    src="scattering_mode_0.png"
    alt="Alt text"
    title=""
    style="display: inline-block; margin: 0 auto; max-width: 200px">
</p>

The scattered field $u_s$ can be decomposed in modes $\mathrm U_n$ defined by (Eq. 1):

$$
\tag{7}
u _ s(\mathbf{r}) =  \sum _ {n=-\infty}^{+\infty} \mathfrak{F} _ n\mathrm U _ n(k\mathbf{r}), \qquad \text{where}\quad \mathfrak{F} _ n := \sum _ {n=-\infty}^{+\infty} \sum _ {j=1}^J \mathrm{V} _ {n'-n}(-k\mathbf{r} _ i )f_{n'}^i
$$

The coefficients $\mathfrak F_n$ depend on the particles configuration, which centers are given by $\mathbf r_1,\dots,\mathbf r_J$. In the above, the coefficients $f_{n}^i$ are obtained after solving the Foldy-Lax equations. The coefficients $\mathfrak F_n$ for $n\in [0,N]$, N = basis_field_order, are computed as follows:

In [ ]:
basis_field_order = 4;                                    # number of modes to compute
F = mode_analysis(input_mode, ω, host_medium, sp_MC;
                radius_big_cylinder=radius_big_cylinder, 
                basis_order=basis_order, 
                basis_field_order=basis_field_order,
                nb_iterations=1);

abs_F = [abs(f[1]) for f in F]
scatter(0:basis_field_order,abs_F,label=false,markerstrokewidth=.5,markersize=7,markershape=:dtriangle)

<p align="center">
    <img
    src="deterministic_modal_decomposition_mode_0.png"
    alt="Alt text"
    title=""
    style="display: inline-block; margin: 0 auto; max-width: 200px">
</p>

#### 3.1.c) Average scattered field over particles configurations

The previous steps can be repeated in order to compute the averaged scattered field over several particles configurations, denoted $\langle u_s \rangle(\mathbf{r})$. 

$$
\tag{8}
\langle u_s \rangle(\mathbf{r}) =  \sum_{n=-\infty}^{+\infty} \langle \mathfrak{F}_n\rangle \mathrm U_n(k\mathbf{r})
$$

The averaged scattered field is computed as follows:

In [ ]:
using Statistics
x_vec, _ = points_in_shape(region;resolution=100);                            # space discretization 
nb_of_configurations = 200;                                                   # total number of config 
A = complex(zeros(length(x_vec),nb_of_configurations));                       # store the fields of each config

for i=1:nb_of_configurations
    particles = renew_particle_configurations(sp_MC,radius_big_cylinder);     # renew config
    sim = FrequencySimulation(particles,source);
    us = run(sim,x_vec,[ω];only_scattered_waves=true,basis_order=basis_order) # solve for #specific config
    A[:,i] = mean.(us.field[:,1]) 
end 
mean_A  = mean(A,dims=2);
mean_us = FrequencySimulationResult(mean_A,x_vec,[ω]);

# plot average scattered field
plot(mean_us,ω; field_apply=real,seriestype = :contour,c=:balance) 

<p align="center">
    <img
    src="mean_us_mode_0.png"
    alt="Alt text"
    title=""
    style="display: inline-block; margin: 0 auto; max-width: 200px">
</p>


The empirical average of $\langle \mathfrak{F}_n\rangle$ obtained with 200 configurations is computed by specifying nb_iterations=200 in the function mode_analysis:

In [ ]:
F = mode_analysis(input_mode, ω, host_medium, sp_MC;
                radius_big_cylinder=radius_big_cylinder, 
                basis_order=basis_order, 
                basis_field_order=basis_field_order,
                nb_iterations=nb_of_configurations);

F_average = mean.(F)
scatter(0:basis_field_order,abs.(F_average),label=false,markerstrokewidth=.5,markersize=7,markershape=:dtriangle)
scatter!(xlabel="n",ylabel=L"$\langle\mathfrak{F}_n\rangle$")
scatter!(title="modes for one $(nb_of_configurations) realisations")

<p align="center">
    <img
    src="average_modal_decomposition_mode_0.png"
    alt="Alt text"
    title=""
    style="display: inline-block; margin: 0 auto; max-width: 200px">
</p>

We can see from this previous plot that the empirical average $\langle\mathfrak F_n\rangle\approx 0 $ if $n\neq0$ and this is an expected result for a source of the form (6): the resulting scattered field is also radial:

$$
\tag{9}
u_i(\mathbf r) = \mathrm V_0(k\mathbf r)
\implies
\langle u_s\rangle(\mathbf r) = \langle\mathfrak F_0\rangle\mathrm U_0(k\mathbf r)
$$

#### 3.1.d) Computation of $\mathrm T_0$

The radial source (6) corresponds to the choice $g_n=\delta_{n,0}$ in (2). From (3), we see that $\langle u_s\rangle(\mathbf r)$ then coincide with (9), furthermore we have that $\mathrm T_0 = \langle\mathfrak F_0\rangle$. The computation of this quantity doesn't require to compute the whole scattered field $u_s(\mathbf r)$ but only the coefficient $\mathfrak F_0$ provided by (7) with $n=0$. $\langle\mathfrak F_0\rangle$ is then computed by averaging several realisations of $\mathfrak F_0$ over particle configurations.

### 3.2) Modal scattering

The result (9) is valid for other modal sources: for any $n$,

$$
\tag{10}
u_i(\mathbf r) = \mathrm V_n(k\mathbf r)
\implies
\langle u_s\rangle(\mathbf r) = \langle\mathfrak F_n\rangle\mathrm U_n(k\mathbf r)
$$

The radial case corresponded to the case $n=0$, we illustrate this result again with the choice $n=3$ (change imput_mode $= 3$ instead of $0$ in the above). We obtain the following results:


<p align="center">
  <img src="source_mode_3.png" width="400" />
</p>




<p align="center">
  <img src="scattering_mode_3.png" width="400" />
  <img src="mean_us_mode_3.png" width="400" /> 
</p>

<p align="center">
  <img src="deterministic_modal_decomposition_mode_3.png" width="400" />
  <img src="average_modal_decomposition_mode_3.png" width="400" /> 
</p>

The coefficient $\langle\mathfrak F_n\rangle$ appearing in (10) corresponds to  $\mathrm T_n$. This quantity is computed with (7) and averaged other particles configurations. The function run_MC_validation! uses this modal scattering property to compute the coefficients of the effective T-matrix.